In [42]:
import requests
import aco_lib

In [43]:
taxi_type = 'green'
year = '2020'

# download_file(f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{taxi_type}/{taxi_type}_tripdata_{year}-01.csv.gz', f'data/raw/{taxi_type}/{year}')

aco_lib.download_file(f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/{taxi_type}/{taxi_type}_tripdata_{year}-01.csv.gz', f'../data/raw/{taxi_type}/{year}')

'green_tripdata_2020-01.csv.gz'